In [1]:
import tensorflow as tf

print(tf.__version__)

1.9.0


In [2]:
def layer(input, weight_shape, bias_shape):
    weight_std = (2.0 / weight_shape[0]) ** 0.5
    w_init = tf.random_normal_initializer(stddev=weight_std)
    b_init = tf.constant_initializer(value=0)
    W = tf.get_variable(name="W", shape=weight_shape, initializer=w_init)
    b = tf.get_variable(name="b", shape=bias_shape, initializer=b_init)
    return tf.nn.relu(tf.matmul(input, W) + b)

In [3]:
def inference(x):
    with tf.variable_scope("hidden_1"):
        hidden_1 = layer(x, [784, 256], [256])
        
    with tf.variable_scope("hidden_2"):
        hidden_2 = layer(hidden_1, [256, 256], [256])
    
    with tf.variable_scope("output"):
        output_ = layer(hidden_2, [256, 10], [10])
        output = tf.identity(output_, name="output")
        
    return output

In [4]:
def loss(output, y):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y)
    loss = tf.reduce_mean(cross_entropy)
    return loss

In [5]:
def training(cost, global_step):
    tf.summary.scalar("cost", cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_opt = optimizer.minimize(cost, global_step=global_step)
    return train_opt

In [6]:
def evaluate(output, y, name):
    compare = tf.equal(tf.argmax(output, axis=1), tf.argmax(y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(compare, tf.float32))
    tf.summary.scalar("eval/{}".format(str(name)), accuracy)
    return accuracy

In [7]:
full_training = False
learning_rate = 1e-2
training_epochs = 1000 if full_training else 100
batch_size = 100 if full_training else 10
display_step = 1000 if full_training else 20

In [8]:
# uncomment the below line to download the data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
with tf.Graph().as_default():
    with tf.variable_scope("mlp_model"):
        x = tf.placeholder("float", [None, 784], name="input")   # x is batch input
        y = tf.placeholder("float", [None, 10])   # y is output for 10 classification

        output = inference(x)   # get the inference result
        cost = loss(output=output, y=y)   # get the loss
        global_step = tf.Variable(0, name="global_step", trainable=False)   # training step
        train_opt = training(cost=cost, global_step=global_step)   # training body
        eval_opt_val = evaluate(output=output, y=y, name="valid")   # evaluation result
        eval_opt_test = evaluate(output=output, y=y, name="test")   # evaluation result

        init_var = tf.global_variables_initializer()
        summary_opt = tf.summary.merge_all()   # merge summary in each epoch
        saver = tf.train.Saver()   # for saving checkpoints

        with tf.Session() as sess:
            # write the summary 
            summary_writer = tf.summary.FileWriter("./log", graph=sess.graph)
            sess.run(init_var)   # initialize all variables

            for epoch in range(training_epochs):
                avg_cost = 0.
                total_batch = int(mnist.train.num_examples / batch_size)

                for idx in range(total_batch):
                    batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)   # get the batch data

                    feed_dict_data = {x: batch_x, y: batch_y}
                    sess.run(train_opt, feed_dict=feed_dict_data)   # feed into data and start training

                    batch_cost = sess.run(cost, feed_dict=feed_dict_data)
                    avg_cost += batch_cost / total_batch   # calculate the average loss

                if epoch % display_step == 0:
                    # record log
                    feed_dict_val_data = {x: mnist.validation.images, y: mnist.validation.labels}
                    acc = sess.run(eval_opt_val, feed_dict=feed_dict_val_data)   # calculate the accuracy

                    print("Accuracy:", acc, "Vaildation Error:", (1-acc))
                    tf.summary.scalar("validation_accuracy", acc)  

                    summary_str = sess.run(summary_opt, feed_dict=feed_dict_data)
                    summary_writer.add_summary(summary_str, sess.run(global_step))   # write out the summary

                    saver.save(sess, "./log/model-ckpt", global_step=global_step)

            print("Training finishing.")

            feed_dict_test_data = {x: mnist.test.images, y: mnist.test.labels}
            acc = sess.run(eval_opt_test, feed_dict=feed_dict_test_data)   # test result
            print("Test Accuracy:",acc)

Accuracy: 0.7514 Vaildation Error: 0.248600006104
Accuracy: 0.7828 Vaildation Error: 0.217199981213
Accuracy: 0.7844 Vaildation Error: 0.215600013733
Accuracy: 0.7844 Vaildation Error: 0.215600013733
Accuracy: 0.7836 Vaildation Error: 0.216400027275
Training finishing.
Test Accuracy: 0.7848
